In [1]:
import gymnasium as gym
import pygame
import os
import numpy as np
from stable_baselines3 import DQN, DDPG
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback, CallbackList
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.noise import NormalActionNoise
import tensorboard

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


# Continous

In [2]:
# DDPG
learning_rate = 0.001
batch_size = 64
discount_factor = 0.99
total_timesteps = 250000

In [3]:
models_dir = "models/DDPG"
logs_dir = "logs"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

In [4]:
eval_env = gym.make(
    'LunarLander-v2',
    continuous = True,
)

checkpoint_callback = CheckpointCallback(save_freq=10000,
                                         save_path=models_dir,
                                         name_prefix='DDPG_model',
                                         )

eval_callback = EvalCallback(eval_env,
                            best_model_save_path=models_dir,
                            log_path = logs_dir,
                            eval_freq=100,
                            )

callback = CallbackList([checkpoint_callback, eval_callback])

In [5]:
env_continuous = gym.make(
    'LunarLander-v2',
    continuous = True,
)

env_continuous.reset()

(array([ 0.00635777,  1.4082228 ,  0.64394754, -0.11990228, -0.00736018,
        -0.14586368,  0.        ,  0.        ], dtype=float32),
 {})

In [6]:
# Define the action noise for DDPG
n_actions = env_continuous.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

In [7]:
model_DDPG = DDPG('MlpPolicy', env_continuous, 
                learning_rate=learning_rate, 
                batch_size=batch_size, 
                gamma=discount_factor, 
                learning_starts=50000, 
                verbose=1,
                device='cuda',
                tensorboard_log=logs_dir)

NameError: name 'env' is not defined

In [ ]:
model_DDPG.learn(total_timesteps=total_timesteps, callback=callback ,tb_log_name = "DDPG")

In [ ]:
env_continuous.close()

In [ ]:
episodes = 2
env = gym.make(
    'LunarLander-v2',
    continuous = True,
    render_mode = 'human',
)

num_actions = env.action_space.shape[-1]

# Define the path to the model
model_DDPG_path = os.path.join("models", "DDPG", "best_model")

# Load the model 
try:
    model_DDPG = DDPG.load(model_DDPG_path)
    print("Model loaded successfully.")
except FileNotFoundError:
    print(f"Model file not found at path: {model_DDPG_path}. Please check the path.")


for episode in range(0, episodes):
    state, info = env.reset()
    # state = state[0]
    terminated = False
    score = 0

    while not terminated :
        action, _states = model_DDPG.predict(state, deterministic=True)
        state, reward, terminated , truncated, info  = env.step(action)
        score += reward
    
    print(score)


# Close the Pygame window
pygame.quit()

Model loaded successfully.
Current working directory: c:\Users\msaee\Desktop\Semester 10\RL\Quizzes\Quiz 3\Material\DQN
